# Init

In [ ]:
# convenience Import for all LabOne Q Functionality
from laboneq.simple import *

from IPython.display import clear_output  # clears the output of a cell upon call
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

# automatically pretty-prints instances of LabOne Q classes
from rich import pretty
pretty.install()

# LabOne Q
from laboneq.contrib.example_helpers.plotting import plot_helpers as plt_hlp
from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor

# Experiment library
from laboneq_applications.experiments import experiment_library as exp_lib
from laboneq_applications.experiments import readout_calibration_experiments as ro_calib_exp
from laboneq_applications.experiments import qubit_calibration_experiments as qb_calib_exp
from laboneq_applications import loading_helpers as load_hlp
from laboneq_applications import calibration_helpers as calib_hlp
from laboneq_applications.experiments import quantum_operations as qt_ops
from laboneq_applications.analysis import analysis_helpers as ana_hlp
from laboneq_applications.analysis import cal_trace_rotation as cal_tr_rot
from laboneq.analysis import fitting as fit_mods

In [ ]:
emulate = True

In [ ]:
DEFAULT_ACQUISITION_COUNT = 2**6  # use 2**12 for more accurate tune-up
DEFAULT_FREQ_SWEEP_STEPS = 101  # use 501 for more accurate tune-up

Choose the directory where the measurement data is saved.

In [ ]:
# Set the directory where you want the library to save your data"
# example: directory = r'C:\Users\data'
data_directory = r'data'  # creates a folder names "data" in the current folder
data_directory

# Create measurement_setup and qubits

## Create from scratch

In [ ]:
# PSI measurement_setup
from laboneq.dsl.device import create_connection
from laboneq.dsl.device.instruments import HDAWG, SHFQA, SHFQC, SHFSG, PQSC
measurement_setup = DeviceSetup("TopFeedlineT6C")
measurement_setup.add_dataserver(host="localhost", port="8004")
measurement_setup.add_instruments(
    HDAWG(uid="device_hdawg", address="dev8793"),
    SHFQC(uid="device_shfqc", address="dev12250"),
    PQSC(uid="device_pqsc", address="dev10091"),
)
for i, qbn in enumerate(['qb1', 'qb2', 'qb3']): # these names must match the qubit names in the qubit_parameters.json!
    measurement_setup.add_connections(
        "device_hdawg",
        create_connection(to_signal=f"{qbn}/flux_line", ports=f"SIGOUTS/{i+1}"),
    )

    measurement_setup.add_connections(
        "device_shfqc",
        create_connection(to_signal=f"{qbn}/drive_line", ports=f"SGCHANNELS/{str(i)}/OUTPUT"),
        create_connection(to_signal=f"{qbn}/drive_line_ef", ports=f"SGCHANNELS/{str(i)}/OUTPUT"),
        create_connection(to_signal=f"{qbn}/measure_line", ports="QACHANNELS/0/OUTPUT"),
        create_connection(to_signal=f"{qbn}/acquire_line", ports="QACHANNELS/0/INPUT"),
    )
measurement_setup.add_connections(
    "device_pqsc",
    create_connection(to_instrument="device_hdawg", ports="ZSYNCS/2"),
    create_connection(to_instrument="device_shfqc", ports="ZSYNCS/0"),
)

In [ ]:
# Three transmon qubits
# load qubit parameters from a json file
qubit_parameters = load_hlp.load_qubit_parameters_from_json(
    full_filepath='qubit_parameters.json')
qb1, qb2, qb3 = calib_hlp.create_qubits_from_parameters(qubit_parameters, measurement_setup)

In [ ]:
qubits = [qb1, qb2, qb3]
measurement_setup.qubits = qubits

In [ ]:
# Set two integration kernels for the qubits, in order to pass the 3-state active reset tests
for qubit in qubits:
    qubit.parameters.readout_integration_kernels = [
        pulse_library.const(uid=f"{qubit.uid}_weights_1", length=qubit.parameters.readout_integration_length, amplitude=1),
        pulse_library.const(uid=f"{qubit.uid}_weights_2", length=qubit.parameters.readout_integration_length, amplitude=1)
    ]
    qubit.parameters.readout_integration_kernels_type = "default"  # use just one kernel

## Or load from a previous experiment folder produced by the library

In [ ]:
# # takes the latest experiment direcotry in data_directory
# folder = load_hlp.get_latest_experiment_directory(data_directory)
# print(folder)
# # or choose it yourself
# # example: folder = data_directory + r'\20231219\165746_Rabi_active_reset_ge_qb1'
# folder = data_directory + 'rest of the path to the experiment directory'
# print(folder)

In [ ]:
# measurement_setup = load_hlp.load_measurement_setup_from_experiment_directory(folder)
# qb1, qb2, qb3 = measurement_setup.qubits
# qubits = qb1, qb2, qb3

## DC-source 

In [ ]:
import logging
mylogger = logging.getLogger("user_func")
def set_dc_bias(session, voltage, qubit):
    slot = qubit.parameters.dc_slot
    if slot not in [13, 14, 15, 17, 18]:
        raise ValueError("ZI is currenly using slots 13, 14, 15, 17 and 18. Please do not use other people's slots!")
    mylogger.info(f"Called 'user_func' with params: voltage={voltage:.1f}")
    print(f'Setting QDac slot {slot} to {voltage} V ... ')
    return

## Connect to session

In [ ]:
# Create and connect to a session
session = Session(device_setup=measurement_setup)
session.register_neartime_callback(set_dc_bias)  # register the set_dc_bias function as neartime callback
session.connect(do_emulation=emulate, reset_devices=False)

# Signal Propagation Delay

In [ ]:
qubit = qb1
qubit_temporary_values = [
    (qubit, "reset_delay_length", 1e-6),
]
sweep_parameters_dict = {
    qubit.uid: [LinearSweepParameter(f'delay_{qubit.uid}', 0, 1e-6, 5, 'Signal Delay Time, $\\tau$ ($\\mu$s)')]}
acquisition_metainfo = dict(count=2**0)
analysis_metainfo = dict()#show_figures=True)
experiment_metainfo = dict()
exp = ro_calib_exp.SignalPropagationDelay([qubit], session, measurement_setup,
                                         acquisition_metainfo=acquisition_metainfo,
                                         sweep_parameters_dict=sweep_parameters_dict,
                                         analysis_metainfo=analysis_metainfo,
                                         qubit_temporary_values=qubit_temporary_values,
                                         data_directory=data_directory, do_analysis=True,
                                         save=True, run=True)

# Spectroscopy

## Resonator spec - qubits

In [ ]:

qubits_to_measure = [qb2]
qubit_temporary_values = []
sweep_parameters_dict = {}
for qubit in qubits_to_measure:
    qubit_temporary_values += [
        (qubit, "reset_delay_length", 1e-6),
    ]

    freqs = qubit.parameters.readout_resonator_frequency + np.linspace(-50e6, 80e6, 201)
    sweep_parameters_dict[qubit.uid] = [
        SweepParameter(f'freq_{qubit.uid}',  freqs,  'Readout Frequency, $f_{RO}$ (GHz)')
    ]

acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
experiment_metainfo = dict(pulsed=True)
analysis_metainfo = dict(show_figures=True, measurement_type="reflection",
                         fit_lorentzian=True,
                         fit_complex=False)
exp = ro_calib_exp.ResonatorSpectroscopy(qubits_to_measure, session, measurement_setup,
                                         experiment_metainfo=experiment_metainfo,
                                         acquisition_metainfo=acquisition_metainfo,
                                         analysis_metainfo=analysis_metainfo,
                                         sweep_parameters_dict=sweep_parameters_dict,
                                         qubit_temporary_values=qubit_temporary_values,
                                         do_analysis=True,  # whether to run analysis
                                         update=False,  # whether or not to update the qubit parameters after data analysis
                                         save=False,  # whether to save the results and measurement setup
                                         run=True,  # whether to automatically run the experiment upon executing this cell
                                         data_directory=data_directory)

In [ ]:
# If the cell above was executed with run = False, then that just instantiates the ResonatorSpectroscopy class.
# The experiment itself has not been defined, configured, or complied. We can do all these steps by hand for full control

# To run the experiment manually, we can call:
exp.autorun()

# Alternatively, we can execute all the steps in autorun by hand:
exp.save_measurement_setup()  # creates a new folder in data_directory and saves the measurement_setup
exp.save_experiment_metainfo() # saves the experiment metainfo as a pickle file
exp.define_experiment()  # defines the sweep, sections and pulses
exp.configure_experiment()  # configures the calibration
exp.compile_experiment()  # compiles the experiment
exp.run_experiment()  # execute the experiment
exp.execute_exit_condition()  # an exit condition such as resetting the voltage of the dc source
exp.analyse_experiment()  # analyse the experiment
exp.update_entire_setup()  # update the qubit parameters and the measurement setup based on the analysis results
exp.save_results()  # save the results object
exp.save_analysis_results()  # save the fit results (if revelant; nothing is fitted for resonator spectroscopy)

In [ ]:
# The Labone Q Experiment instance is stored in
# exp.experiment

In [ ]:
# Access the Results instance
# exp.results

In [ ]:
# Access the analysis results
exp.analysis_results

## Resonator spec - wide sweep


In [ ]:

qubit = qb1
qubit_temporary_values = [
    (qubit, "reset_delay_length", 1e-6),
]
sweep_parameters_dict = {
    qubit.uid: [LinearSweepParameter(f'freq_{qubit.uid}', -500e6, 500e6, 1001, 'IF Frequency, $f_{IF}$ (GHz)'),
                LinearSweepParameter(f'lo_freq_{qubit.uid}', 6e9, 8e9, 3, 'LO Frequency, $f_{LO}$ (GHz)')]}
# try this with concatenation of ranges for the lo sweep!
acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
analysis_metainfo = dict(show_figures=True)
experiment_metainfo = dict(neartime_sweep_parameter="frequency")
exp = ro_calib_exp.ResonatorSpectroscopy([qubit], session, measurement_setup,
                                         acquisition_metainfo=acquisition_metainfo,
                                         sweep_parameters_dict=sweep_parameters_dict,
                                         analysis_metainfo=analysis_metainfo,
                                         qubit_temporary_values=qubit_temporary_values,
                                         experiment_name='ResonatorSpectroscopyFullRange',
                                         data_directory=data_directory, do_analysis=True, save=True, run=True)

## Resonator spec - voltage sweep

In [ ]:
qubits_to_measure = [qb3]
qubit_temporary_values = []
sweep_parameters_dict = {}
for qubit in qubits_to_measure:
    qubit_temporary_values += [
        (qubit, "reset_delay_length", 1e-6),
    ]


    # choose frequency sweep range
    freqs = qubit.parameters.readout_resonator_frequency + np.linspace(-50e6, 80e6, 201)

    # choose voltage sweep range
    voltages = qubit.parameters.dc_voltage_parking + np.linspace(-1, 1, 11)
#     voltages = np.linspace(0, -2, 15)

    # define sweep parameters
    sweep_parameters_dict[qubit.uid] = [
        SweepParameter(f"freq_{qubit.uid}", freqs, "Readout Frequency, $f_{RO}$ (GHz)"),
        SweepParameter(f"dc_volt_sweep_{qubit.uid}", voltages, "DC Voltage, $V$ (V)")
    ]

acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
# neartime_sweep_parameter must correspond to the first input par of neartime_callback_function after session
experiment_metainfo = dict(
    neartime_sweep_parameter="voltage",
    neartime_callback_function=set_dc_bias)
analysis_metainfo = dict(find_peaks=False, show_figures=True, sweet_spot_to_update="lss")
exp = ro_calib_exp.ResonatorSpectroscopy(qubits_to_measure, session, measurement_setup,
                                         experiment_metainfo=experiment_metainfo,
                                         acquisition_metainfo=acquisition_metainfo,
                                         analysis_metainfo=analysis_metainfo,
                                         sweep_parameters_dict=sweep_parameters_dict,
                                         qubit_temporary_values=qubit_temporary_values,
                                         apply_exit_condition=True,
                                         data_directory=data_directory, update=False,
                                         do_analysis=True, save=True, run=True)

## Resonator spec - amp sweep

In [ ]:
qubits_to_measure = [qb2]
qubit_temporary_values = []
sweep_parameters_dict = {}
for qubit in qubits_to_measure:
    qubit_temporary_values += [
        (qubit, "reset_delay_length", 1e-6),
        (qubit, "readout_range_out", -5),
    ]

    freqs = qubit.parameters.readout_resonator_frequency + np.linspace(-50e6, 80e6, 201)
    sweep_parameters_dict[qubit.uid] = [
        SweepParameter(f'freq_{qubit.uid}', freqs, "Readout Frequency, $f_{RO}$ (GHz)"),
        SweepParameter(f"ro_amplitude_{qubit.uid}", np.arange(0.1, 1.1, 0.1), "Readout Amplitude Scaling")
    ]

acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
# neartime_sweep_parameter must correspond to the first input par of neartime_callback_function after session
experiment_metainfo = dict(neartime_sweep_parameter="amplitude", pulsed=False)
analysis_metainfo = dict(show_figures=True)
exp = ro_calib_exp.ResonatorSpectroscopy(qubits_to_measure, session, measurement_setup,
                                         experiment_metainfo=experiment_metainfo,
                                         acquisition_metainfo=acquisition_metainfo,
                                         analysis_metainfo=analysis_metainfo,
                                         sweep_parameters_dict=sweep_parameters_dict,
                                         qubit_temporary_values=qubit_temporary_values,
                                         data_directory=data_directory, do_analysis=True,
                                         save=True, run=True)

## Qubit Spectroscopy

In [ ]:
for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    qubit_temporary_values = []
    sweep_parameters_dict = {}
    analysis_metainfo = dict(show_figures=True)
    for qubit in qubits_to_measure:
        qubit_temporary_values += [
            (qubit, "drive_range", -25),
            (qubit, "reset_delay_length", 1e-6),
            (qubit, "spectroscopy_amplitude", 1),
        ]

        # choose frequency sweep range
        ge_freq = qubit.parameters.resonance_frequency_ge
        freqs = np.linspace(ge_freq - 25e6, ge_freq + 25e6, 101)
        # create sweep parameter
        sweep_parameters_dict[qubit.uid] = [SweepParameter(f'freq_{qubit.uid}',  freqs, 'Qubit Frequency [Hz]')]
    #     analysis_metainfo["frequency_filter_for_fit"] = lambda f: f < ge_freq - 0.04e9

    experiment_metainfo = dict(pulsed=True)
    acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
    exp = qb_calib_exp.QubitSpectroscopy(qubits_to_measure, session, measurement_setup,
                                         experiment_metainfo=experiment_metainfo,
                                         acquisition_metainfo=acquisition_metainfo,
                                         analysis_metainfo=analysis_metainfo,
                                         sweep_parameters_dict=sweep_parameters_dict,
                                         qubit_temporary_values=qubit_temporary_values,
                                         do_analysis=True, update=False,
                                         data_directory=data_directory, save=True, run=True)

In [ ]:
# reanalyse with a frequency filter
analysis_metainfo.update(dict(frequency_filter_for_fit=lambda f: f < 5.75e9))
exp.analyse_experiment()

## Qubit Spectroscopy - wide sweep

In [ ]:
for qubits_to_measure in [
    [qb1, qb2],
    [qb3]
]:

    # lo freqs must be the same for both qubits if they share lo
    lo_frequencies = LinearSweepParameter('outer_freq', 4e9, 7e9, 4, 'LO Frequency, $f_{LO}$ (GHz)')
    sweep_parameters_dict = {}
    for qubit in qubits_to_measure:
        sweep_parameters_dict[qubit.uid] = [LinearSweepParameter(
        f'freq_{qubit.uid}', -500e6, 500e6, 1001, 'Qubit Frequency, $f_{qb}$ (GHz)'),
                                  lo_frequencies
                                 ]

    experiment_metainfo = dict(pulsed=True, neartime_sweep_parameter="frequency",)
    acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
    analysis_metainfo = dict(show_figures=True)
    exp = qb_calib_exp.QubitSpectroscopy(qubits_to_measure, session, measurement_setup,
                                    experiment_metainfo=experiment_metainfo,
                                    acquisition_metainfo=acquisition_metainfo,
                                    analysis_metainfo=analysis_metainfo,
                                    sweep_parameters_dict=sweep_parameters_dict,
                                    do_analysis=True, update=False,
                                    data_directory=data_directory, save=True, run=True)

## Qubit Spectroscopy - voltage sweep

In [ ]:
qubits_to_measure = [qb3]
qubit_temporary_values = qubit_temporary_values
sweep_parameters_dict = {}
for qubit in qubits_to_measure:
    qubit_temporary_values += [
        (qubit, "reset_delay_length", 1e-6),
    ]

    freqs = qubit.parameters.resonance_frequency_ge + np.linspace(-200e6, 50e6, DEFAULT_FREQ_SWEEP_STEPS)
    voltages = qubit.parameters.dc_voltage_parking + np.linspace(-0.5, 0.5, 11)
    sweep_parameters_dict[qubit.uid] = [
        SweepParameter(f"freq_{qubit.uid}", freqs, "Qubit Frequency, $f_{qb}$ (GHz)"),
        SweepParameter(f"dc_volt_sweep_{qubit.uid}", voltages, "DC Voltage, $V$ (V)")
    ]

acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
# neartime_sweep_parameter must correspond to the first input par of neartime_callback_function after session
experiment_metainfo = dict(
    neartime_sweep_parameter="voltage",
    neartime_callback_function=set_dc_bias,
)
analysis_metainfo = dict(show_figures=True)
exp = qb_calib_exp.QubitSpectroscopy(qubits_to_measure, session, measurement_setup,
                                     experiment_metainfo=experiment_metainfo,
                                     acquisition_metainfo=acquisition_metainfo,
                                     analysis_metainfo=analysis_metainfo,
                                     sweep_parameters_dict=sweep_parameters_dict,
                                     qubit_temporary_values=qubit_temporary_values,
                                     apply_exit_condition=True,
                                     data_directory=data_directory, do_analysis=True,
                                     update=True, save=True, run=True)

##  Qubit Spectroscopy - amp sweep

In [ ]:
for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    qubit_temporary_values = []
    sweep_parameters_dict = {}
    for qubit in qubits_to_measure:
        qubit_temporary_values += [
            (qubit, "reset_delay_length", 1e-6),
        ]

        freqs = qubit.parameters.resonance_frequency_ge + np.linspace(-200e6, 50e6, DEFAULT_FREQ_SWEEP_STEPS)
        sweep_parameters_dict[qubit.uid] = [
            SweepParameter(f"freq_{qubit.uid}", freqs, "Qubit Frequency, $f_{qb}$ (GHz)"),
            SweepParameter(f"spec_amplitude_{qubit.uid}", np.arange(0.1, 1.1, 0.1), "Spectroscopy Amplitude Scaling")
        ]

    acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
    # neartime_sweep_parameter must correspond to the first input par of neartime_callback_function after session
    experiment_metainfo = dict(
        neartime_sweep_parameter="amplitude",
    )
    analysis_metainfo = dict(show_figures=True)
    exp = qb_calib_exp.QubitSpectroscopy(qubits_to_measure, session, measurement_setup,
                                         experiment_metainfo=experiment_metainfo,
                                         acquisition_metainfo=acquisition_metainfo,
                                         analysis_metainfo=analysis_metainfo,
                                         sweep_parameters_dict=sweep_parameters_dict,
                                         qubit_temporary_values=qubit_temporary_values,
                                         data_directory=data_directory, do_analysis=True,
                                         save=True, run=True)

# Pulse tune-up

## Rabi

In [ ]:
# Rabi
for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    for transition_to_calibrate in [
        'ge',
        'ef'
    ]:
        for preparation_type in [
            "wait",
            "active_reset"
        ]:
            qubit_temporary_values = []
            sweep_parameters_dict = {}
            for qubit in qubits_to_measure:
                qubit_temporary_values += [
                    (qubit, "reset_delay_length", 200e-6),
                ]
                if preparation_type == "active_reset":
                    qubit_temporary_values += [
                        # active reset that includes ef needs two kernels
                        (qubit, "readout_integration_kernels_type", "optimal"),
                        (qubit, "readout_integration_parameters", {
                            "length": 2e-6,
                            "kernels": [{"function": "const"}, {"function": "const"}],
                        }),
                    ]                    

                pi_amp = qubit.parameters.drive_parameters_ef['amplitude_pi'] if transition_to_calibrate == "ef" else \
                    qubit.parameters.drive_parameters_ge['amplitude_pi']
                swp_end = pi_amp + 0.1
                sweep_parameters_dict[qubit.uid] = [
                    LinearSweepParameter(f'amps_{qubit.uid}', 0, swp_end, 21, 'Amplitude Scaling')]

            acquisition_metainfo = dict(count=2 ** 0)
            experiment_metainfo = dict(cal_states=transition_to_calibrate,
                                       transition_to_calibrate=transition_to_calibrate,
                                       preparation_type=preparation_type
                                      )
            analysis_metainfo = dict(show_figures=True, do_fitting=False)
            exp = qb_calib_exp.AmplitudeRabi(qubits_to_measure, session, measurement_setup,
                                             experiment_metainfo=experiment_metainfo,
                                             acquisition_metainfo=acquisition_metainfo,
                                             analysis_metainfo=analysis_metainfo,
                                             sweep_parameters_dict=sweep_parameters_dict,
                                             qubit_temporary_values=qubit_temporary_values,
                                             do_analysis=True, update=False,
                                             data_directory=data_directory, save=False, run=True)

## Ramsey

In [ ]:
# Ramsey
acquisition_metainfo = dict(count=2 ** 0)
for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    for transition_to_calibrate in [
        'ge',
        'ef'
    ]:
        for preparation_type in [
            "wait",
            "active_reset"
        ]:
            qubit_temporary_values = []
            sweep_parameters_dict = {}
            experiment_metainfo = dict(detuning={}, cal_states=transition_to_calibrate,
                                       transition_to_calibrate=transition_to_calibrate,
                                       preparation_type=preparation_type,
                                      )
            analysis_metainfo = dict(show_figures=True, do_fitting=preparation_type != "active_reset")
            for qubit in qubits_to_measure:
                qubit_temporary_values += [
                    (qubit, "reset_delay_length", 200e-6),
                ]
                if preparation_type == "active_reset":
                    qubit_temporary_values += [
                        # active reset that includes ef needs two kernels
                        (qubit, "readout_integration_kernels_type", "optimal"),
                        (qubit, "readout_integration_parameters", {
                            "length": 2e-6,
                            "kernels": [{"function": "const"}, {"function": "const"}],
                        }),
                    ]                  

                if transition_to_calibrate == "ef":
                    sweep_parameters_dict[qubit.uid] = [LinearSweepParameter(f'delays_{qubit.uid}', 0, 25e-6, 41),
                                                        'Pulse Delay, $\\tau$ ($\\mu$s)']
                    experiment_metainfo['detuning'][qubit.uid] = 0.673e6
                else:
                    sweep_parameters_dict[qubit.uid] = [LinearSweepParameter(f'delays_{qubit.uid}', 0, 20e-6, 41),
                                                        'Pulse Delay, $\\tau$ ($\\mu$s)']
                    experiment_metainfo['detuning'][qubit.uid] = 0.473e6

            exp = qb_calib_exp.Ramsey(qubits_to_measure, session, measurement_setup,
                                      experiment_metainfo=experiment_metainfo,
                                      acquisition_metainfo=acquisition_metainfo,
                                      analysis_metainfo=analysis_metainfo,
                                      sweep_parameters_dict=sweep_parameters_dict,
                                      qubit_temporary_values=qubit_temporary_values,
                                      do_analysis=True, update=True,
                                      data_directory=data_directory, save=True, run=True)

## QScale

In [ ]:
# QScale
for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    for transition_to_calibrate in [
        'ge',
        'ef'
    ]:
        for preparation_type in ["wait", "active_reset"]:
            qubit_temporary_values = []
            sweep_parameters_dict = {}
            for qubit in qubits_to_measure:
                qubit_temporary_values += [
                    (qubit, "reset_delay_length", 200e-6),
                ]
                if preparation_type == "active_reset":
                    qubit_temporary_values += [
                        # active reset that includes ef needs two kernels
                        (qubit, "readout_integration_kernels_type", "optimal"),
                        (qubit, "readout_integration_parameters", {
                            "length": 2e-6,
                            "kernels": [{"function": "const"}, {"function": "const"}],
                        }),
                    ]                  
                beta = qubit.parameters.drive_parameters_ef['beta'] if transition_to_calibrate == "ef" else \
                    qubit.parameters.drive_parameters_ge['beta']
            #     beta = 0

                beta_values = beta + np.linspace(-0.05, 0.05, 13)
                sweep_parameters_dict[qubit.uid] = [SweepParameter(f'qscales_{qubit.uid}', beta_values,
                                                                   "Quadrature Scaling, $q$")]

            experiment_metainfo = dict(cal_states=transition_to_calibrate,
                                       transition_to_calibrate=transition_to_calibrate,
                                       preparation_type=preparation_type,
                                      )
            acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
            analysis_metainfo = dict(show_figures=True, do_fitting=False)
            exp = qb_calib_exp.QScale(qubits_to_measure, session, measurement_setup,
                                      experiment_metainfo=experiment_metainfo,
                                      acquisition_metainfo=acquisition_metainfo,
                                      sweep_parameters_dict=sweep_parameters_dict,
                                      qubit_temporary_values=qubit_temporary_values,
                                      analysis_metainfo=analysis_metainfo,
                                      do_analysis=True, update=False,
                                      data_directory=data_directory, save=True, run=True)

## T1

In [ ]:
# T1
for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    for transition_to_calibrate in [
        'ge',
        'ef'
    ]:
        for preparation_type in ["wait", "active_reset"]:
            qubit_temporary_values = []
            sweep_parameters_dict = {}
            for qubit in qubits_to_measure:
                qubit_temporary_values += [
                    (qubit, "reset_delay_length", 1e-6),
                ]
                if preparation_type == "active_reset":
                    qubit_temporary_values += [
                        # active reset that includes ef needs two kernels
                        (qubit, "readout_integration_kernels_type", "optimal"),
                        (qubit, "readout_integration_parameters", {
                            "length": 2e-6,
                            "kernels": [{"function": "const"}, {"function": "const"}],
                        }),
                    ]                  
                sweep_parameters_dict[qubit.uid] = [LinearSweepParameter(f'delays_{qubit.uid}', 0, 50e-6, 31),
                                                   "Pulse Delay, $\\tau$ ($\\mu$s)"]
            experiment_metainfo = dict(cal_states=transition_to_calibrate,
                                       transition_to_calibrate=transition_to_calibrate,
                                       preparation_type=preparation_type,
                                      )
            acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
            analysis_metainfo = dict(show_figures=True, do_fitting=preparation_type != "active_reset")
            exp = qb_calib_exp.T1(qubits_to_measure, session, measurement_setup,
                                  experiment_metainfo=experiment_metainfo,
                                  acquisition_metainfo=acquisition_metainfo,
                                  sweep_parameters_dict=sweep_parameters_dict,
                                  analysis_metainfo=analysis_metainfo,
                                  qubit_temporary_values=qubit_temporary_values,
                                  do_analysis=True, data_directory=data_directory,
                                  save=True, run=True)

## Echo

In [ ]:
# Echo
acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    for transition_to_calibrate in [
        'ge',
        'ef'
    ]:
        for preparation_type in ["wait", "active_reset"]:
            qubit_temporary_values = []
            sweep_parameters_dict = {}
            experiment_metainfo = dict(detuning={}, cal_states=transition_to_calibrate,
                                       transition_to_calibrate=transition_to_calibrate,
                                       preparation_type=preparation_type,
                                      )
            analysis_metainfo = dict(show_figures=True, do_fitting=preparation_type != "active_reset")
            for qubit in qubits_to_measure:
                qubit_temporary_values += [
                    (qubit, "reset_delay_length", 1e-6),
                    (qubit, "readout_pulse_length", 0.2e-6),
                    (qubit, "readout_integration_length", 0.2e-6),
                ]
                if preparation_type == "active_reset":
                    qubit_temporary_values += [
                        # active reset that includes ef needs two kernels
                        (qubit, "readout_integration_kernels_type", "optimal"),
                        (qubit, "readout_integration_parameters", {
                            "length": 2e-6,
                            "kernels": [{"function": "const"}, {"function": "const"}],
                        }),
                    ]                  
                sweep_parameters_dict[qubit.uid] = [LinearSweepParameter(f'delays_{qubit.uid}', 0, 30e-6, 41),
                                                    "Pulse Delay, $\\tau$ ($\\mu$s)"]
                experiment_metainfo['detuning'][qubit.uid] = 0.473e6
            exp = qb_calib_exp.Echo(qubits_to_measure, session, measurement_setup,
                                    experiment_metainfo=experiment_metainfo,
                                    acquisition_metainfo=acquisition_metainfo,
                                    analysis_metainfo=analysis_metainfo,
                                    sweep_parameters_dict=sweep_parameters_dict,
                                    qubit_temporary_values=qubit_temporary_values,
                                    do_analysis=True, data_directory=data_directory,
                                    save=True, run=True)

# Ramsey Parking

In [ ]:
# Ramsey
acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
experiment_metainfo = dict(
    neartime_callback_function=set_dc_bias,
    detuning={})

for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    for preparation_type in [
        "wait",
        "active_reset"
    ]:
        experiment_metainfo["preparation_type"] = preparation_type

        qubit_temporary_values = []
        sweep_parameters_dict = {}
        for qubit in qubits_to_measure:
            qubit_temporary_values += [
                    (qubit, "reset_delay_length", 200e-6),
            ]
            if preparation_type == "active_reset":
                qubit_temporary_values += [
                    # active reset that includes ef needs two kernels
                    (qubit, "readout_integration_kernels_type", "optimal"),
                    (qubit, "readout_integration_parameters", {
                        "length": 2e-6,
                        "kernels": [{"function": "const"}, {"function": "const"}],
                    }),
                ]              
            voltages = qubit.parameters.dc_voltage_parking + np.linspace(-0.05, 0.05, 5)
            sweep_parameters_dict[qubit.uid] = [LinearSweepParameter(f'delays_{qubit.uid}', 0, 200e-9, 41),
                                                SweepParameter(f"dc_volt_sweep_{qubit.uid}", voltages, 'DC Voltage (V)')]
            experiment_metainfo['detuning'][qubit.uid] = 41.73e6

        analysis_metainfo = dict(show_figures=True, do_fitting=preparation_type != "active_reset")
        exp = qb_calib_exp.RamseyParking(qubits_to_measure, session, measurement_setup,
                                         experiment_metainfo=experiment_metainfo,
                                         acquisition_metainfo=acquisition_metainfo,
                                         analysis_metainfo=analysis_metainfo,
                                         sweep_parameters_dict=sweep_parameters_dict,
                                         qubit_temporary_values=qubit_temporary_values,
                                         do_analysis=True, update=False,
                                         data_directory=data_directory, save=False, run=True)

# Readout tune-up

## Optimal readout frequency

In [ ]:
for qubits_to_measure in [
    [qb2]
]:
    qubit_temporary_values = []
    sweep_parameters_dict = {}
    for qubit in qubits_to_measure:
        qubit_temporary_values += [
            (qubit, "reset_delay_length", 1e-6),
        ]
        freqs = qubit.parameters.readout_resonator_frequency + np.linspace(-80e6, 80e6, 301)
        sweep_parameters_dict[qubit.uid] = [SweepParameter(f'freq_{qubit.uid}',  freqs,  'Readout Frequency, $f_{RO}$ (GHz)')]

    acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
    experiment_metainfo = dict()#pulsed=True, acquisition_type=AcquisitionType.INTEGRATION)
    analysis_metainfo = dict(show_figures=True)
    exp = ro_calib_exp.DispersiveShift(qubits_to_measure, session, measurement_setup,
                                       experiment_metainfo=experiment_metainfo,
                                       acquisition_metainfo=acquisition_metainfo,
                                       analysis_metainfo=analysis_metainfo,
                                       sweep_parameters_dict=sweep_parameters_dict,
                                       qubit_temporary_values=qubit_temporary_values,
                                       preparation_states=('g', 'e'),
                                       do_analysis=True, update=False,
                                       data_directory=data_directory, save=True, run=True)

## StateDiscrimination

## Optimal integration kernel

In [ ]:
for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    for preparation_states in [
        ("g", "e"),
        ("g", "e", "f")
    ]:
        for preparation_type in [
            "wait", 
            # TODO: "active_reset" -- analysis fails with active_reset in tests
        ]:
            qubit_temporary_values = []
            for qubit in qubits_to_measure:
                qubit_temporary_values += [
                    (qubit, "reset_delay_length", 2e-6),
                ]
                if preparation_type == "active_reset":
                    qubit_temporary_values += [
                        # active reset that includes ef needs two kernels
                        (qubit, "readout_integration_kernels_type", "optimal"),
                        (qubit, "readout_integration_parameters", {
                            "length": 2e-6,
                            "kernels": [{"function": "const"}, {"function": "const"}],
                        }),
                    ]                  
            acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
            analysis_metainfo = dict(show_figures=True)
            experiment_metainfo = dict(
                preparation_type=preparation_type
            )
            if preparation_type == "active_reset":
                for num_resets in [1, 10]:
                    # test repeated active reset
                    experiment_metainfo["num_resets"] = num_resets
                    exp = ro_calib_exp.StateDiscrimination(
                        qubits_to_measure, session, measurement_setup,
                        experiment_metainfo=experiment_metainfo,
                        acquisition_metainfo=acquisition_metainfo,
                        analysis_metainfo=analysis_metainfo,
                        preparation_states=preparation_states,
                        qubit_temporary_values=qubit_temporary_values,
                        do_analysis=True, update=False,
                        data_directory=data_directory, save=True, run=True)
            else:
                exp = ro_calib_exp.StateDiscrimination(
                    qubits_to_measure, session, measurement_setup,
                    experiment_metainfo=experiment_metainfo,
                    acquisition_metainfo=acquisition_metainfo,
                    analysis_metainfo=analysis_metainfo,
                    preparation_states=preparation_states,
                    qubit_temporary_values=qubit_temporary_values,
                    do_analysis=True, update=False,
                    data_directory=data_directory, save=True, run=True)                

In [ ]:
for qubits_to_measure in [
    [qb2]
]:
    qubit_temporary_values = []
    for qubit in qubits_to_measure:
        qubit_temporary_values += [
            (qubit, "reset_delay_length", 200e-6),
        ]
    acquisition_metainfo = dict(count=DEFAULT_ACQUISITION_COUNT)
    experiment_metainfo = dict()
    analysis_metainfo = dict(show_figures=True)
    exp = ro_calib_exp.OptimalIntegrationKernels(
        qubits_to_measure, session, measurement_setup,
        experiment_metainfo=experiment_metainfo,
        acquisition_metainfo=acquisition_metainfo,
        analysis_metainfo=analysis_metainfo,
        qubit_temporary_values=qubit_temporary_values,
        do_analysis=True, update=False,
        data_directory=data_directory, save=False, run=True)

# New experiment

In [ ]:
exp = exp_lib.ExperimentTemplate([qb1], session, measurement_setup,
                                 experiment_name='MyCoolExperiment',
                                 signals=['drive', 'flux', 'measure', 'acquire'],
                                 data_directory=data_directory, save=True)

In [ ]:
# exc_sec = Section(uid='excitation')
# exc_sec.play(signal=exp.signal_name('drive', qb1), pulse=qt_ops.quantum_gate(qb1, 'X180_ge'))
# measure_acquire_sec = exp.create_measure_acquire_sections(qb1, play_after='excitation')

# exp.create_acquire_rt_loop()
# exp.experiment.add(exp.acquire_loop)
# exp.acquire_loop.add(exc_sec)
# exp.acquire_loop.add(measure_acquire_sec)

In [ ]:
# exp_rabi = exp.experiment

# exp.experiment.sections = []
# amplitude_sweep = LinearSweepParameter('amp', 0, 1, 31)
# ## define Rabi experiment pulse sequence
# # outer loop - real-time, cyclic averaging
# with exp_rabi.acquire_loop_rt(
#     uid="rabi_shots",
#     count=2**10,
#     averaging_mode=AveragingMode.CYCLIC,
#     acquisition_type=AcquisitionType.INTEGRATION,
# ):
#     # inner loop - real time sweep of Rabi ampitudes
#     with exp_rabi.sweep(uid="rabi_sweep", parameter=amplitude_sweep):
#         # play qubit excitation pulse - pulse amplitude is swept
#         with exp_rabi.section(
#             uid="qubit_excitation", alignment=SectionAlignment.RIGHT
#         ):
#             exp_rabi.play(
#                 signal="drive_qb1", pulse=qt_ops.quantum_gate(qb1, 'X180_ge'),
#                 amplitude=amplitude_sweep
#             )
#         # readout pulse and data acquisition
#         with exp_rabi.section(uid="readout_section", play_after="qubit_excitation"):
#             readout_pulse = qt_ops.readout_pulse(qb1)
#             # play readout pulse on measure line
#             exp_rabi.play(signal="measure_qb1", pulse=readout_pulse)
#             # trigger signal data acquisition
#             exp_rabi.acquire(
#                 signal="acquire_qb1",
#                 handle="amp_rabi",
#                 kernel=readout_pulse,
#             )
#         # relax time after readout - for qubit relaxation to groundstate and signal processing
#         with exp_rabi.section(uid="reserve", length=1e-6):
#             exp_rabi.reserve(signal="measure_qb1")

In [ ]:
# exp.autorun()